# Deliverable Notebook

In [3]:
import pandas as pd
import re
pd.set_option('display.max_rows', None)

Drug Classes:
- ALLERGY, SYSTEMIC & NASAL
- DERMATOLOGICS
- GI PRODUCTS
- IMMUNOLOGY
- MULTIPLE SCLEROSIS
- NERVOUS SYSTEM DISORDERS
- ONCOLOGICS
- OPHTHALMOLOGY, GENERAL
- OTHER CNS
- OTHER HAEMATOLOGICALS
- VACCINES (PURE, COMB, OTHER)

In [1]:
drug_class = 'ALLERGY, SYSTEMIC & NASAL'

In [20]:
# Retrieving all data about the drug class
data = pd.read_csv('../Processed_Data/drugs_pct_changes_monthly.csv')
data = data[data['Major Class'] == drug_class]
data.Date = pd.to_datetime(data.Date, format='%Y_%m')

# Only NDCs that have full period
last_period = data[(data.Year == 2020) & (data.Month == 7)].NDC.unique()
first_period = data[(data.Year == 2014) & (data.Month == 8)].NDC.unique()
selected_NDC = list(set(last_period).intersection(first_period)) 
data = data[data.NDC.isin(selected_NDC)]

# Updating NDC that do not have LOE date to a future LOE date
data.loc[data['Estimated LOE Date'] == 'Unspecified', 'Estimated LOE Date'] = 'May-24'

# Getting rid of NDC that are too close to their LOE date
data['Estimated LOE Date'] = pd.to_datetime(data['Estimated LOE Date'], format='%b-%y')
data = data[data['Estimated LOE Date'] >= pd.to_datetime('2020-07-01')]

# Retrieving year over year percent change data
yy_data = pd.read_csv('../Year Over Year/df_yearoveryear_class.csv')
yy_data.Date = pd.to_datetime(yy_data.Date, format='%Y_%m')

# Retrieving computed predictions
prophet_1 = pd.read_csv(f'../Predictive Models/Results/Method 1/{drug_class}.csv')
prophet_1 = prophet_1.rename(columns={'Unnamed: 0':'date', 'Number':'# price changes prediction', 'Percentage of drugs change price':'Percent # price changes prediction'})
prophet_1.date = pd.to_datetime(prophet_1.date)

prophet_2 = pd.read_csv(f'../Predictive Models/Results/Method 2/{drug_class}.csv')
prophet_2.date = pd.to_datetime(prophet_2.date)

In [21]:
# Extracting number of unique drug per month
nb_drugs = data.groupby('Date')['NDC'].nunique().to_frame(name='# drugs').reset_index().rename(columns={'Date': 'date'})

# Extracting year over year percent change in Sales weighted WAC
yy_change = yy_data[yy_data['Major Class'] == drug_class][['Date', 'Class_wght_pct_change_y/y']].reset_index(drop=True).rename(columns={'Date': 'date', 'Class_wght_pct_change_y/y': 'Y/Y Percent change'})

# Extracting number of changes per month
nb_price_change = data.groupby('Date')['Changed'].sum().to_frame(name='# price changes').reset_index().rename(columns={'Date': 'date'})

# Helper function
def weigthed_average(data, quantity, weights):
    try:
        return (data[quantity] * data[weights]).sum() / data[weights].sum()
    except ZeroDivisionError:
        return data[quantity]    
    
# Extracting Sales weighted WAC per month
data['Sales'] = data['WAC'] * data['TRx']
sales_wac = data.groupby('Date').apply(weigthed_average, 'WAC', 'Sales').to_frame(name = 'Sales weighted WAC').reset_index().rename(columns={'Date': 'date'})

# Extracting TRx weighted WAC per month
trx_wac = data.groupby('Date').apply(weigthed_average, 'WAC', 'TRx').to_frame(name = 'TRx weighted WAC').reset_index().rename(columns={'Date': 'date'})

# Extracting Prophet 2 predictions
prophet_2 = prophet_2[['date', 'yhat']].rename(columns={'yhat': 'Sales weighted WAC predictions'})

# Merging all and formatting
deliverable = nb_drugs.merge(yy_change, how='outer', on='date')
deliverable = deliverable.merge(nb_price_change, how='outer', on='date')
deliverable = deliverable.merge(sales_wac, how='outer', on='date')
deliverable = deliverable.merge(trx_wac, how='outer', on='date')
deliverable = deliverable.merge(prophet_1, how='outer', on='date')
deliverable = deliverable.merge(prophet_2, how='outer', on='date')

# Formatting date
deliverable['Year'] = deliverable.date.dt.year
deliverable['Month'] = deliverable.date.dt.month
deliverable.drop('date', axis=1, inplace=True)

# Creating handy class column
deliverable['Class'] = drug_class

# Computing percentage nb of drugs that have price change
deliverable['Percent # price changes'] = deliverable['# price changes'] / deliverable['# drugs']

# Cleaning year 2014
deliverable = deliverable[deliverable.Year != 2014]

# Creating actual + prediction column
deliverable['Sales weighted WAC actual predictions'] = deliverable['Sales weighted WAC'].fillna(deliverable['Sales weighted WAC predictions'])

# Creating date column - handier
deliverable['Date'] = deliverable.Year.astype(str) + '-' + deliverable.Month.astype(str)
deliverable['Date'] = pd.to_datetime(deliverable.Date)

# Cleaning order column
deliverable = deliverable[['Class', 'Date', 'Year', 'Month', 'Y/Y Percent change', '# drugs','# price changes', '# price changes prediction', 'Percent # price changes', 'Percent # price changes prediction',  'TRx weighted WAC', 'Sales weighted WAC', 'Sales weighted WAC predictions', 'Sales weighted WAC actual predictions']]


In [22]:
deliverable

,Class,Date,Year,Month,Y/Y Percent change,# drugs,# price changes,# price changes prediction,Percent # price changes,Percent # price changes prediction,TRx weighted WAC,Sales weighted WAC,Sales weighted WAC predictions,Sales weighted WAC actual predictions
5,"VACCINES (PURE, COMB, OTHER)",2015-01-01,2015,1,NaN,42.0,0.0,0.0,0.000000,0.000000,181.647703,211.979795,240.613581,211.979795
6,"VACCINES (PURE, COMB, OTHER)",2015-02-01,2015,2,NaN,42.0,0.0,0.0,0.000000,0.000000,182.768568,217.957136,240.026982,217.957136
7,"VACCINES (PURE, COMB, OTHER)",2015-03-01,2015,3,NaN,42.0,3.0,2.0,0.071429,0.046875,196.235034,231.375602,234.830320,231.375602
8,"VACCINES (PURE, COMB, OTHER)",2015-04-01,2015,4,NaN,42.0,0.0,0.0,0.000000,0.000000,195.698141,231.924880,226.819326,231.924880
9,"VACCINES (PURE, COMB, OTHER)",2015-05-01,2015,5,NaN,42.0,0.0,0.0,0.000000,0.000000,190.960344,228.860520,216.931874,228.860520
10,"VACCINES (PURE, COMB, OTHER)",2015-06-01,2015,6,NaN,42.0,2.0,2.0,0.047619,0.046154,188.190005,226.286949,216.657028,226.286949
11,"VACCINES (PURE, COMB, OTHER)",2015-07-01,2015,7,NaN,42.0,0.0,0.0,0.000000,0.000000,184.242014,224.009493,212.067283,224.009493
12,"VACCINES (PURE, COMB, OTHER)",2015-08-01,2015,8,0.090608,42.0,0.0,0.0,0.000000,0.000000,188.442203,230.133032,225.140170,230.133032
13,"VACCINES (PURE, COMB, OTHER)",2015-09-01,2015,9,0.102192,42.0,0.0,0.0,0.000000,0.000000,225.205772,259.633697,262.472435,259.633697
14,"VACCINES (PURE, COMB, OTHER)",2015-10-01,2015,10,0.106534,42.0,13.0,16.0,0.309524,0.371429,240.232997,268.674221,274.780650,268.674221


In [23]:
deliverable.to_csv(f'deliverable_{drug_class}.csv', index=False)